In [ ]:

"""
transcript format:
{
    "video_id": "4pa9O5cr398",
    "timestamp": "2025-04-06T19:14:11.761808",
    "status": "success",
    "error_type": null,
    "error_message": null,
    "transcripts": [
        {
            "language": "English (auto-generated)",
            "language_code": "en",
            "is_generated": true,
            "data": [
                {
                    "text": "hello",
                    "start": 9.28,
                    "duration": 2.88
                },
                {
...
                {
                    "text": "you",
                    "start": 723.12,
                    "duration": 2.08
                }
            ]
        }
    ]
}
"""

In [2]:
import pandas as pd
import os
import json
import re
import numpy as np
from tqdm import tqdm
from datetime import datetime
from typing import List, Dict, Any, Tuple
from collections import defaultdict

sponsor_dataset = r"C:\Users\caotr\Downloads\transcript_data\sponsorTimes.parquet"
video_info_dataset = r"C:\Users\caotr\Downloads\transcript_data\videoInfo.csv"
transcript_folder = r"C:\Users\caotr\Downloads\transcript_data\transcripts"

In [9]:
# đầu tiên, mở thử sponsor dataset và xem cấu trúc của nó
sponsor_df = pd.read_parquet(sponsor_dataset)
sponsor_df.head()
# dataset khá lớn nên mở mất khoảng 20s


,videoID,startTime,endTime,votes,locked,incorrectVotes,UUID,userID,timeSubmitted,views,category,actionType,service,videoDuration,hidden,reputation,shadowHidden,hashedVideoID,userAgent,description
0,1rb3bMvDdX4,588.94100,631.89777,159,0,1,28aff450-a372-11e9-b256-cb886cabe693,38e7c2af-09f4-4492-bf49-75e443962ccd,1564088876715,4642,sponsor,skip,YouTube,0.0,0,0.0,0,95e409452186a56331b7a58d518361285e18b8db50de20...,None,None
1,fBxtS9BpVWs,41.00000,53.00000,115,0,1,b2465943-1313-449c-b75c-08b14756ac0a,38e7c2af-09f4-4492-bf49-75e443962ccd,1564088876715,776,sponsor,skip,YouTube,0.0,0,0.0,0,bdd81b2b8192683242fe3608c45d5b958ddc71e9b2981a...,None,None
2,9P6rdqiybaw,488.52150,542.11035,-2,0,1,81024780-a367-11e9-b256-cb886cabe693,38e7c2af-09f4-4492-bf49-75e443962ccd,1564088876715,25661,sponsor,skip,YouTube,552.0,0,0.0,0,cc9cd26ee245cb89f2be13d047de8ea1a642c8f56bcb6e...,None,None
3,ulCdoCfw-bY,487.50198,547.48750,-2,0,1,16090680-a367-11e9-b256-cb886cabe693,38e7c2af-09f4-4492-bf49-75e443962ccd,1564088876715,26984,sponsor,skip,YouTube,0.0,0,0.0,0,177779136cde894988da5e2d3160ef38d302a8554d710d...,None,None
4,uqKGREZs6-w,475.52167,532.20874,302,0,1,622f9270-a2a1-11e9-b210-99c885575bb9,38e7c2af-09f4-4492-bf49-75e443962ccd,1564088876715,18060,sponsor,skip,YouTube,0.0,1,0.0,0,70f8d0e75affa202ab510bd86828080af1dddf9218be46...,None,None


In [ ]:
"""
videoID	startTime	endTime	votes	locked	incorrectVotes	UUID	userID	timeSubmitted	views	category	actionType	service	videoDuration	hidden	reputation	shadowHidden	hashedVideoID	userAgent	description
0	1rb3bMvDdX4	588.94100	631.89777	159	0	1	28aff450-a372-11e9-b256-cb886cabe693	38e7c2af-09f4-4492-bf49-75e443962ccd	1564088876715	4642	sponsor	skip	YouTube	0.0	0	0.0	0	95e409452186a56331b7a58d518361285e18b8db50de20...	None	None
1	fBxtS9BpVWs	41.00000	53.00000	115	0	1	b2465943-1313-449c-b75c-08b14756ac0a	38e7c2af-09f4-4492-bf49-75e443962ccd	1564088876715	776	sponsor	skip	YouTube	0.0	0	0.0	0	bdd81b2b8192683242fe3608c45d5b958ddc71e9b2981a...	None	None
2	9P6rdqiybaw	488.52150	542.11035	-2	0	1	81024780-a367-11e9-b256-cb886cabe693	38e7c2af-09f4-4492-bf49-75e443962ccd	1564088876715	25661	sponsor	skip	YouTube	552.0	0	0.0	0	cc9cd26ee245cb89f2be13d047de8ea1a642c8f56bcb6e...	None	None
3	ulCdoCfw-bY	487.50198	547.48750	-2	0	1	16090680-a367-11e9-b256-cb886cabe693	38e7c2af-09f4-4492-bf49-75e443962ccd	1564088876715	26984	sponsor	skip	YouTube	0.0	0	0.0	0	177779136cde894988da5e2d3160ef38d302a8554d710d...	None	None
4	uqKGREZs6-w	475.52167	532.20874	302	0	1	622f9270-a2a1-11e9-b210-99c885575bb9	38e7c2af-09f4-4492-bf49-75e443962ccd	1564088876715	18060	sponsor	skip	YouTube	0.0	1	0.0	0	70f8d0e75affa202ab510bd86828080af1dddf9218be46...	None	None
"""

In [7]:
# hãy kiểm tra các thông số cơ bản của dataset này
print(sponsor_df.info())
# ta thấy có đến hơn 16 triệu dòng dữ liệu và lưu với kiểu parquet mà vẫn tốn đến hơn 2.5GB


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16493888 entries, 0 to 16493887
Data columns (total 20 columns):
 #   Column          Dtype  
---  ------          -----  
 0   videoID         object 
 1   startTime       float64
 2   endTime         float64
 3   votes           int64  
 4   locked          int64  
 5   incorrectVotes  int64  
 6   UUID            object 
 7   userID          object 
 8   timeSubmitted   int64  
 9   views           int64  
 10  category        object 
 11  actionType      object 
 12  service         object 
 13  videoDuration   float64
 14  hidden          int64  
 15  reputation      float64
 16  shadowHidden    int64  
 17  hashedVideoID   object 
 18  userAgent       object 
 19  description     object 
dtypes: float64(4), int64(7), object(9)
memory usage: 2.5+ GB
None


In [10]:
sponsor_df.describe()
# Phân tích thời gian của sponsor, ta thấy:
# - startTime: trung bình bắt đầu ở xấp xỉ phút thứ 15
# - endTime: trung bình kết thúc ở xấp xỉ phút thứ 16
# - thời lượng trung bình khoảng 50s
# có giá trị âm trong startTime và endTime? (có thể lỗi dữ liệu, cần phân tích thêm)
# giá trị lớn nhất cực cao (444 ngày), đây có thể là lỗi dữ liệu, cần phân tích thêm

# Phân tích phản hồi của người dùng:
# - votes: trung bình khoảng 0.276 votes, khá thấp
# số votes dao động từ -501 đến 9994, tức là có phản hồi tích cực và tiêu cực
# 75% dữ liệu có 0 votes, nghĩa là phần lớn segments không có phản hồi

#video info:
# mặc dù info video trong sponsor dataset có thể không cần thiết, nhưng vẫn có vài thông tin có ích
# - trung bình mỗi video nhận được 474 lượt xem (các video này có nhãn sponsor, nên lượt xem trung bình này có thể lớn so với các video bình thường)
# - thời lượng vidoe trung bình khoảng 36 phút, có thể do sponsor nên thời lượng lớn hơn
# - có video với thời lượng 293 ngày, đây có thể là lỗi dữ liệu

# các nhãn đặc biệt:
# - locked: đoạn sponsor này đã bị khóa (có thể đã được kiểm chứng), không thể thay đổi.
# Có khoảng 2.7% bị khóa
# - hidden: đoạn sponsor này đã bị ẩn đi (có thể chất lượng kém)
# Có khoảng 23% bị ẩn, chúng ta có thể drop các dòng này
# - shadowHidden: đoạn sponsor này đã bị ẩn đi (có thể để phân tích?, lưu trữ)
# có khoảng 5.1% bị shadowHidden, có thể drop luôn các dòng này

,startTime,endTime,votes,locked,incorrectVotes,timeSubmitted,views,videoDuration,hidden,reputation,shadowHidden
count,1.649389e+07,1.649389e+07,1.649389e+07,1.649389e+07,1.649389e+07,1.649389e+07,1.649389e+07,1.649389e+07,1.649389e+07,1.649383e+07,1.649389e+07
mean,9.234681e+02,9.732066e+02,2.764275e-01,2.726919e-02,1.000000e+00,1.674681e+12,4.744585e+02,2.153992e+03,2.303449e-01,2.065722e+00,5.149107e-02
std,1.707526e+04,1.959693e+04,8.939685e+00,1.628668e-01,8.877898e-04,3.779940e+10,4.193451e+03,8.296067e+03,6.308490e-01,6.274483e+00,2.209972e-01
min,-2.440000e+02,-6.400000e+01,-5.010000e+02,0.000000e+00,0.000000e+00,1.564089e+12,0.000000e+00,0.000000e+00,0.000000e+00,-2.500000e+00,0.000000e+00
25%,1.614000e+01,5.075400e+01,0.000000e+00,0.000000e+00,1.000000e+00,1.652769e+12,1.000000e+00,3.850000e+02,0.000000e+00,0.000000e+00,0.000000e+00
50%,1.910490e+02,2.363866e+02,0.000000e+00,0.000000e+00,1.000000e+00,1.669894e+12,2.200000e+01,8.620000e+02,0.000000e+00,0.000000e+00,0.000000e+00
75%,7.591000e+02,7.991320e+02,0.000000e+00,0.000000e+00,1.000000e+00,1.705500e+12,1.650000e+02,1.969221e+03,0.000000e+00,0.000000e+00,0.000000e+00
max,3.837856e+07,3.840068e+07,9.994000e+03,1.000000e+00,3.000000e+00,1.742605e+12,1.876532e+06,2.533528e+07,2.000000e+00,2.700000e+01,1.000000e+00


In [11]:
# hãy kiểm tra các giá trị null trong dataset này
print(sponsor_df.isnull().sum())
# lượng null videoID là rất nhỏ, có thể drop
# reputation null có thể chuyển về 0
# userAgent và description có thể drop

videoID                 54
startTime                0
endTime                  0
votes                    0
locked                   0
incorrectVotes           0
UUID                     0
userID                   0
timeSubmitted            0
views                    0
category                 0
actionType               0
service                  0
videoDuration            0
hidden                   0
reputation              58
shadowHidden             0
hashedVideoID            0
userAgent          3275790
description       16268791
dtype: int64


In [13]:
# bắt đầu với việc kiểm tra có bao nhiêu unique videoID trong dataset này
print(sponsor_df["videoID"].nunique())
# như vậy chúng ta có 7704124 videoID khác nhau trong dataset này

7704124


In [ ]:
# các segment của video có thể thuộc vào 4 trường hợp sau:
# - gán thời gian sai nên được gán lại: nhận biết qua thời gian submit, overlap, votes
# - nhiều phân đoạn khác nhau: các segment là của những phân đoạn sponsor khác nhau,
# nhận biết qua các segment xa nhau rõ ràng, không trùng lắp
# - segment bị ẩn do chất lượng kém (hidden): trường hidden, shadowHidden = 1
# - segment trái chiều: người dùng không đồng thuận với đoạn sponsor này
# Video có segment overlap (gán lại vì sai): ~25-30%
# Video có segment riêng biệt (nhiều quảng cáo): ~45-50%
# Video có segment bị ẩn: ~15%
# Video có segment với vote trái chiều: ~5-10%
# Các trường hợp khác: ~5%

In [14]:
# # hãy vẽ biểu phân phối của các cột numeric
# import matplotlib.pyplot as plt
# import seaborn as sns

# def plot_distribution(df: pd.DataFrame, column: str):
#     plt.figure(figsize=(10, 6))
#     sns.histplot(df[column], bins=50, kde=True)
#     plt.title(f'Distribution of {column}')
#     plt.xlabel(column)
#     plt.ylabel('Frequency')
#     plt.show()

# # vẽ biểu phân phối cho các cột numeric
# numeric_columns = sponsor_df.select_dtypes(include=[np.number]).columns.tolist()
# for column in numeric_columns:
#     plot_distribution(sponsor_df, column)
# # vẽ biểu phân phối cho cột votes
# plot_distribution(sponsor_df, 'votes')

In [ ]:
def process_transcripts(transcript_folder, sponsor_dataset):
    # Đọc dữ liệu sponsor
    sponsor_df = pd.read_parquet(sponsor_dataset)
    
    # Tạo dictionary để lưu thông tin sponsor theo video_id
    sponsor_dict = defaultdict(list)
    for _, row in sponsor_df.iterrows():
        sponsor_dict[row['videoID']].append((row['startTime'], row['endTime']))
    
    all_segments = []
    
    # Đọc và xử lý từng file transcript
    for file_name in tqdm(os.listdir(transcript_folder)):
        if file_name.endswith('.json'):
            file_path = os.path.join(transcript_folder, file_name)
            with open(file_path, 'r', encoding='utf-8') as f:
                transcript = json.load(f)
            
            video_id = transcript['video_id']
            
            # Kiểm tra xem transcript có dữ liệu không
            if transcript['status'] != 'success' or not transcript['transcripts']:
                continue
            
            # Lấy dữ liệu transcript
            transcript_data = None
            for t in transcript['transcripts']:
                if t['language_code'] == 'en':
                    transcript_data = t['data']
                    break
            
            if not transcript_data:
                continue
            
            # Xử lý từng đoạn trong transcript
            for segment in transcript_data:
                start_time = segment['start']
                end_time = start_time + segment['duration']
                text = segment['text']
                
                # Xác định xem đoạn này có nằm trong sponsor không
                is_sponsor = 0
                for sponsor_start, sponsor_end in sponsor_dict.get(video_id, []):
                    # Nếu đoạn này nằm chủ yếu trong khoảng sponsor
                    if (start_time >= sponsor_start and start_time < sponsor_end) or \
                       (end_time > sponsor_start and end_time <= sponsor_end) or \
                       (start_time <= sponsor_start and end_time >= sponsor_end):
                        is_sponsor = 1
                        break
                
                all_segments.append({
                    'video_id': video_id,
                    'start': start_time,
                    'end': end_time,
                    'text': text,
                    'is_sponsor': is_sponsor
                })
    
    # Tạo DataFrame từ tất cả các đoạn đã xử lý
    segments_df = pd.DataFrame(all_segments)
    return segments_df

# Gọi hàm xử lý
segments_df = process_transcripts(transcript_folder, sponsor_dataset)

# Lưu DataFrame để sử dụng sau này
segments_df.to_parquet('data_raw/transcript_segments_labeled.parquet', index=False)